# Генерация исключений

In [13]:
from enum import Enum, auto

class HungryCatError(Exception):
    pass

class FeederError(Exception):
    pass

class CloseFeeder(FeederError):
    pass


class NoFood(FeederError):
    def __init__(self, food, message):
        self.food = food
        self.mesage = message


class FeederState(Enum):
    open = auto()
    close = auto()


class Feeder:
    def __init__(self, food):
        super().__init__()
        self.food = food
        self.state = FeederState.close

    def open(self):
        self.state = FeederState.open

    def close(self):
        self.state = FeederState.close

    def get_food(self, food)-> int:
        if self.state != FeederState.open:
            raise CloseFeeder
        if food < self.food:
            self.food -= food
            return food
        raise NoFood(self.food, "Need more food, milord")

In [14]:
class Dog():

    def rob_feeder(self, feeder: Feeder):
        try:
            feeder.get_food(20)
        except CloseFeeder:
            feeder.open()
        except NoFood as no_food:
            food = no_food.food
            feeder.get_food(food)
        finally:
            print("Gav-Gav")




In [15]:
feeder = Feeder(60)
dog = Dog()

In [17]:
dog.rob_feeder(feeder)


Gav-Gav


In [11]:
class Cat():
    def rob_feeder(self, feeder: Feeder):
        try:
            feeder.get_food(10)
        except FeederError:
            raise HungryCatError

In [20]:
feeder = Feeder(60)
cat = Cat()
cat.rob_feeder(feeder)

HungryCatError: 

# Менеджеры контекстов

In [22]:
feeder = Feeder(60)
dog = Dog()
dog.rob_feeder(feeder)

Gav-Gav


In [23]:
feeder.state

<FeederState.open: 1>

In [26]:
from contextlib import contextmanager


@contextmanager
def open(feeder: Feeder):
    try:
        feeder.open()
        yield feeder
    finally:
        feeder.close()

In [27]:
with open(feeder):
    dog.rob_feeder(feeder)

Gav-Gav


In [31]:
class Feeder:
    def __init__(self, food):
        super().__init__()
        self.food = food
        self.state = FeederState.close

    def __enter__(self):
        self.open()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(exc_type, exc_val, exc_tb)
        self.close()

    def open(self):
        self.state = FeederState.open

    def close(self):
        self.state = FeederState.close

    def get_food(self, food)-> int:
        if self.state != FeederState.open:
            raise CloseFeeder
        if food < self.food:
            self.food -= food
            return food
        raise NoFood(self.food, "Need more food, milord")

In [33]:
cat = Cat()
with Feeder(5) as feeder:
    cat.rob_feeder(feeder)

<class '__main__.HungryCatError'>  <traceback object at 0x7f53b73de240>


HungryCatError: 

In [34]:
feeder.state

<FeederState.close: 2>